# Curva Zero-Cupom Brasil - Bootstrapping + NSS

Taxas de títulos públicos calculadas pela Anbima: https://www.anbima.com.br/pt_br/informar/taxas-de-titulos-publicos.htm

Curva de Juros: Gráficos Atualizados: https://clubedospoupadores.com/curva-de-juros

Spread dos Títulos Públicos do Brasil: https://clubedospoupadores.com/curva-de-juros/spread

Curva de Juros – Inflação Implícita: https://clubedospoupadores.com/curva-de-juros/inflacao

seria possivel adicionar um colapse menu por ano?

In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Timestamp
from typing import Union
from datetime import date, datetime
from calendars.daycounts import DayCounts
from calendars.custom_date_types import TODAY
from finmath.termstructure.curve_models import CurveBootstrap, NelsonSiegelSvensson
from finmath.brazilian_bonds.government_bonds import LTN, NTNF
from pathlib import Path

# Add the src directory to the Python module search path
sys.path.append(str(Path().resolve()))
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime

# Widget for selecting the reference date
ref_date_picker = widgets.DatePicker(
    description='Settle date:',
    value=datetime(2025, 6, 19),
    disabled=False
)

run_button = widgets.Button(
    description="Recalcular Curva",
    button_style='success'
)

output = widgets.Output()

display(widgets.HBox([ref_date_picker, run_button]), output)

Output()

In [2]:
def build_and_plot_curve(ref_date):



    # Caminho do Excel
    #file_path = r"C:\Users\tsiqueira4\OneDrive - Bloomberg LP\Desktop\master_thesis_economics\datos_y_modelos\Domestic\domestic_sovereign_curve_brazil.xlsx"
    
    # Sobe até a raiz do repo automaticamente
    repo_root = Path.cwd()
    while not (repo_root / ".git").exists() and repo_root != repo_root.parent:
        repo_root = repo_root.parent

    file_path = repo_root / "datos_y_modelos" / "Domestic" / "domestic_sovereign_curve_brazil.xlsx"


    # -----------------------------
    # CARREGAR DADOS E SEPARAR TÍTULOS
    # -----------------------------

    df = pd.read_excel(file_path)
    df['MATURITY'] = pd.to_datetime(df['MATURITY'])

    # LTNs
    ltn_df = df[df['papel'] == 'LTN'].copy()
    ltn_df = ltn_df[['MATURITY', 'YAS_BOND_YLD']].dropna().sort_values('MATURITY')
    ltn_expires = ltn_df['MATURITY'].dt.date.tolist()
    ltn_yields = (ltn_df['YAS_BOND_YLD'].astype(float) / 100).tolist()

    # NTN-Fs
    ntnf_df = df[df['papel'] == 'NTNF'].copy()
    ntnf_df = ntnf_df[['MATURITY', 'YAS_BOND_YLD']].dropna().sort_values('MATURITY')
    ntnf_expires = ntnf_df['MATURITY'].dt.date.tolist()
    ntnf_yields = (ntnf_df['YAS_BOND_YLD'].astype(float) / 100).tolist()

    # -----------------------------
    # OBJETOS DE TÍTULOS
    # -----------------------------

    ltn_prices, ltn_cash_flows = [], []
    for T, y in zip(ltn_expires, ltn_yields):
        bond = LTN(expiry=T, rate=y, ref_date=ref_date)
        ltn_prices.append(bond.price)
        ltn_cash_flows.append(pd.Series(index=[T], data=[bond.principal]))

    ntnf_prices, ntnf_cash_flows = [], []
    for T, y in zip(ntnf_expires, ntnf_yields):
        bond = NTNF(expiry=T, rate=y, ref_date=ref_date)
        ntnf_prices.append(bond.price)
        ntnf_cash_flows.append(bond.cash_flows)

    # -----------------------------
    # BOOTSTRAP E NSS
    # -----------------------------

    all_prices = ltn_prices + ntnf_prices
    all_cash_flows = ltn_cash_flows + ntnf_cash_flows
    cb = CurveBootstrap(prices=all_prices, cash_flows=all_cash_flows, ref_date=ref_date)
    zero_curve = cb.zero_curve.to_frame('zero')

    dc = DayCounts('bus/252', calendar='cdr_anbima')
    ltn_curve = pd.DataFrame(index=[dc.tf(ref_date, x) for x in ltn_expires],
                             columns=['LTN'], data=ltn_yields)
    ntnf_curve = pd.DataFrame(index=[dc.tf(ref_date, x) for x in ntnf_expires],
                              columns=['NTNF'], data=ntnf_yields)

    nss = NelsonSiegelSvensson(prices=all_prices, cash_flows=all_cash_flows, ref_date=ref_date)
    nss_curve = pd.Series(index=zero_curve.index,
                          data=[nss.rate_for_ytm(betas=nss.betas, ytm=t) for t in zero_curve.index],
                          name='nss')

    curves = pd.concat([zero_curve, ltn_curve, ntnf_curve, nss_curve.to_frame()], axis=1).sort_index()

    # -----------------------------
    # CURVAS INTERPOLADAS
    # -----------------------------

    x_dense = np.linspace(0.01, max(curves.index), 300)
    y_dense_zero = [cb.rate_for_date(t) * 100 for t in x_dense]
    y_dense_nss = [nss.rate_for_ytm(betas=nss.betas, ytm=t) * 100 for t in x_dense]

    # -----------------------------
    # PLOTAGEM
    # -----------------------------

    plt.figure(figsize=(15, 10))

    # Curvas interpoladas
    plt.plot(x_dense, y_dense_zero, label='Curva Zero (Bootstrap - Interpolada)', color='blue')
    plt.plot(x_dense, y_dense_nss, label='Curva NSS (Interpolada)', color='darkorange')

    # Pontos observados
    plt.plot(curves.index, curves['LTN'] * 100, label='LTN Observada', linestyle='--', marker='x', color='green')
    plt.plot(curves.index, curves['NTNF'] * 100, label='NTNF Observada', linestyle='--', marker='s', color='red')

    # Rótulos dos vencimentos
    for i, (x, y) in enumerate(zip(ltn_curve.index, ltn_curve['LTN'])):
        plt.text(x, y * 100 + 0.05, f'LTN\n{ltn_expires[i].strftime("%Y-%m-%d")}', ha='center', fontsize=7)
    for i, (x, y) in enumerate(zip(ntnf_curve.index, ntnf_curve['NTNF'])):
        plt.text(x, y * 100 + 0.05, f'NTNF\n{ntnf_expires[i].strftime("%Y-%m-%d")}', ha='center', fontsize=7)

    # Layout final
    plt.title(f'Curva Zero-Cupom Brasil (Interpolada) — Settle date: {ref_date}', fontsize=20)
    plt.xlabel('Prazo (anos)', fontsize=16)
    plt.ylabel('Taxa (% a.a.)', fontsize=16)
    plt.grid(True)
    plt.legend(fontsize=12)
    plt.tight_layout()
    plt.show()
                 
    # -----------------------------
    # TABELA DE DADOS OBSERVADOS
    # -----------------------------

    ltn_df_display = pd.DataFrame({
        'Tipo': 'LTN',
        'Maturity': ltn_expires,
        'Yield (% a.a.)': [y * 100 for y in ltn_yields],
        'T.Maturity (anos)': [dc.tf(ref_date, d) for d in ltn_expires]
    })

    ntnf_df_display = pd.DataFrame({
        'Tipo': 'NTNF',
        'Maturity': ntnf_expires,
        'Yield (% a.a.)': [y * 100 for y in ntnf_yields],
        'T.Maturity (anos)': [dc.tf(ref_date, d) for d in ntnf_expires]
    })

    tabela = pd.concat([ltn_df_display, ntnf_df_display], ignore_index=True).sort_values('T.Maturity (anos)')

    print("Dados observados utilizados para as curvas:")
    display(tabela.style.format({"Yield (% a.a.)": "{:.4f}", "T.Maturity (anos)": "{:.4f}"}))



In [3]:
def on_run_clicked(b):
    with output:
        clear_output(wait=True)
        ref_date = ref_date_picker.value
        build_and_plot_curve(ref_date)

run_button.on_click(on_run_clicked)